## 0 Preparing Data

We first generate images for training using ```trdg``` package. Using the code ```trdg -i words.txt -c 20000 --output_dir data/train```; it's the same for testing

Then we create a custom PyTorch dataset object, which has ```__len__``` and ```__getitem__``` methods. We also specify the path to the dataset and apply transform to it. The transforms are grayscaling, normalizing, and converting to tensor.

In [ ]:
import os
import torch
import numpy as np



from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import sampler
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

dtype = torch.float32 # using float 32 throughout
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

We first define the ```MyCollator()``` function which does the padding horizontally for our images and forming a batch, which is used for ```collate_fn``` in the ```DataLoader``` object in PyTorch. 

In [ ]:
def MyCollator(batch):
        '''
        Parameters:
            -batch: list of tuples with dictionary {'img':Image,'idx':Index}, where img is a tensor
                    of the shape C * H * W
        '''
        width = [item['img'].shape[2] for item in batch] # list of widths of images C H W

        # print(max(width)) you can check if collate is working properly
        
        indices = [item['idx'] for item in batch] # list of indices

        imgs = torch.ones([len(batch),batch[0]['img'].shape[0],\
            batch[0]['img'].shape[1], max(width)],dtype=dtype) # create array of ones for padding
        
        for idx, item in enumerate(batch):
            try:
                # fit image into the array generated, only altering the width
                # by replacing ones with the original but keeping extras in new
                imgs[idx,:,:,0:width[idx]] = item['img']
            except:
                print(f'{imgs.shape} \t ERROR \n')
        # Forming the batch
        item = {'img':imgs,'idx':indices}

        if 'label' in batch[0].keys():
            # if label is present then also make it a list and attach to batch
            labels = [item['label'] for item in batch]
            item['labels'] = labels
        
        return item

Now we define the dataset with an option of specifying the train or test data path; note how we tell ```__getitem__``` to return a dictionary of image, indices, and labels.

In [ ]:
class MyDset(Dataset):
    '''Create own dataset as subclass of Dataset.'''
    def __init__(self,opt={'path':'data','imgdir':'\train'}):
        '''
        Initialize MyDset object.
        Parameters:
            -opt: a dictionary of strings indicating the location/path
        '''
        super(MyDset,self).__init__()
        
        self.path = os.path.join(opt['path'],opt['imgdir']) # To the directory storing the images

        self.images = os.listdir(self.path) # List the images name

        self.nsamp = len(self.images) 

        f = lambda x: os.path.join(self.path,x) # Join the path and the name
        self.imagepaths = list(map(f,self.images))

        self.transform = transforms.Compose([\
            transforms.Grayscale(1),\
                transforms.ToTensor(),\
                    transforms.Normalize((0.5,), (0.5,))\
                        ]) # applying transforms to the images

        # self.collate_fn = SynthCollator # specified mean to form batches
    def __len__(self):
        '''Return length of dataset.'''
        return self.nsamp
    
    def __getitem__(self, index):
        '''Return single tensor label pair'''
        imagepath = self.imagepaths[index]
        name = os.path.basename(imagepath) # return final component of path
        img = Image.open(imagepath)

        if self.transform is not None: # transform images
            img = self.transform(img)

        item = {'img': img, 'idx': index} # item to return
        item['label'] = name.split('_')[0] # the word in the image # image is named in name_0x0x0x this way
        return item

We now create a ```DataLoader``` object from the dataset defined above

In [ ]:
dset_train = MyDset(opt={'path':'data/images','imgdir':'train'})
dset_test  = MyDset(opt={'path':'data/images','imgdir':'test'})

collate_func = MyCollator

NUM_TRAIN =int(len(dset_train)*0.9)

# Define the training and its validation dataset
loader_train = DataLoader(dset_train,\
    batch_size = 20,\
        collate_fn = collate_func,\
            sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

loader_val = DataLoader(dset_train,\
    batch_size = 20,\
        collate_fn=collate_func,\
            sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, len(dset_train))))

loader_test = DataLoader(dset_test,\
    batch_size = 20,\
        collate_fn=collate_func,\
            sampler=sampler.SubsetRandomSampler(range(1000)))

We can try to print out to see if `collate_fn` is working properly, each batch is of size $20$, and the widths of the images are the same

In [ ]:
for x in loader_train:
    # print(x)
    # print(x['img'].shape)
    break

We try to visualize some of the images. Note the images have not been padded to the same width, as we have not applied the ```collate_fn``` until the ```DataLoader``` stage.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(1,3*3+1):
    img = dset_train[i]['img'][0,:,:]
    print(img.shape)
    fig.add_subplot(3,3,i)
    plt.imshow(img)
    plt.axis('off')
plt.show()


# 1 Implementing Model

We will now implement the model in Pytorch as a Lightning Module.  


The network consists of consecutive convolutional layers with max-pooling layers of $2\times 2$ and $2 \times 1$; the conv layers produce feature maps as column vectors, which act as the sequential input for the RNN. (Include paper)

We first deal with the RNN, which is bidirectional and uses the LSTM structure. The code is based on [this repo](https://github.com/meijieru/crnn.pytorch/tree/master/models)

In [ ]:
import pytorch_lightning as pl

In [ ]:
class BLR(nn.Module):
    '''Implement the bidirectional LSTM RNN.'''
    def __init__(self,nIn,nHidden,nOut):
        '''
        Initialize the BLR.
        Parameters:
            -nIn: input size
            -nHidden: hidden size
            -nOut: output size
        '''
        super().__init__()
        self.rnn = nn.LSTM(nIn,nHidden,\
            bidirectional=True)
        # final FC necessary??
        self.linear = nn.Linear(2*nHidden,nOut) # bi-direction

    def forward(self,input):
        # https://discuss.pytorch.org/t/why-do-we-need-flatten-parameters-when-using-rnn-with-dataparallel/46506
        self.rnn.flatten_parameters()

        r,_ = self.rnn(input) # output of LSTM is out,(hidden,cell)
        L, N, H = r.size() # getting sequence len, batch size, and hidden size*2

        r_alt = r.view(L*N,H)
        output = self.linear(r_alt)

        output = output.view(L,N,-1) # out is now a sequence of L each term a vector of size N
        return output

We test our implementation of ```BLR```.

In [ ]:
def testBLR():
    '''Test output size of BLR.'''
    model = BLR(512,16,26)
    input = torch.zeros((25,64,512)) # 100*32 image, batch_size=64, 512 final features
    output = model(input)
    return output.size()

testBLR() # Expect L*N*class

Now we implement the full CRNN as in the paper. The structure is as follows:  

$3 \times 3 $ ```Conv```, $64$ feature maps  
$2 \times 2 $ ```MaxPool2D```, stride = $2$, img_size halved  

$N \times H \times W = 64 \times 16 \times 64$  

$3 \times 3 $ ```Conv```, $128$ feature maps  
$2 \times 2 $ ```MaxPool2D```, stride = $2$, img_size halved   

$N \times H \times W = 128 \times 8 \times 32$  

$3 \times 3 $ ```Conv```, $256$ feature maps  
$3 \times 3 $ ```Conv```, $256$ feature maps  (can be omitted?)

$2 \times 1 $ ```MaxPool2D```, stride = $2$, img_size halved vertically  

$N \times H \times W = 256 \times 4 \times 16$  

$3 \times 3 $ ```Conv```, $512$ feature maps    

$BatchNorm$  Layer.

$2 \times 1 $ ```MaxPool2D```, stride = $2$, img_size halved vertically  

$N \times H \times W = 512 \times 2 \times 16$  

$2 \times 2 $ ```Conv```, $512$ feature maps, img_size halved vertically 

$N \times H \times W = 512 \times 1 \times 16$  

We define a ```Conv``` layer function for convenience.

In [ ]:
def myConv(ker_size=3,in_chan=1,out_chan=64,\
    nPad=1, batchNorm=False, leaky=False,label=00):
    '''
    Implement custom conv layer.
    Parameters:
        -ker_size: int, size of 
        -in_chan: int, the input size
        -out_chan: int, the output size
        -nPad: tuple or 
        -batchNorm: bool
        -maxPool: bool
        -label: to distinguish different layers
    Output:
        -A list of nn objects
    '''
    out = nn.Sequential()
    conv_layer = nn.Conv2d(in_channels=in_chan,out_channels=out_chan,\
        kernel_size=ker_size,padding=nPad,stride=1)
        
    out.add_module('conv{0}'.format(label),conv_layer)
    
    if batchNorm:
        out.add_module('bn{0}'.format(label),nn.BatchNorm2d(out_chan))
    if leaky:
        out.add_module('leaky{0}'.format(label),nn.LeakyReLU(0.2, inplace=True))
    else:
        out.add_module('relu{0}'.format(label),nn.ReLU(inplace=True))
    
    return out


Now we define the network as a `LightningModule`, including the **CTC** loss function and `forward` for testing. We use some helper functions.


In [ ]:
from utils import labelConverter

In [ ]:
class CRNN(pl.LightningModule):
    '''
    Implement the CRNN.
    Parameters:
        -opt: a dictionary containing the paramters
            -leaky: boolean for ReLU
            -nHidden: int for no. of RNN layers
            -nClass: int for class of characters
        opt = {'leaky':False,'nHidden':256,'nClass':26,'lr':1e-3})
    '''
    def __init__(self, \
        opt = {'leaky':False,'nHidden':256,'nClass':26, 'lr':1e-3}):
        
        super().__init__()

        self.converter = labelConverter("abcdefghijklmnopqrstuvwxyz")
        self.lr = opt['lr']

        leaky = opt['leaky']
        nHidden = opt['nHidden']
        nClass = opt['nClass']

        group = nn.Sequential()

        group.add_module('conv1',myConv(leaky=leaky))
        group.add_module('mpool1',nn.MaxPool2d(2,2))

        group.add_module('conv2',myConv(3,64,128,1,leaky=leaky))
        group.add_module('mpool2',nn.MaxPool2d(2,2))

        group.add_module('conv3',myConv(3,128,256,1,leaky=leaky))
        group.add_module('mpool3',nn.MaxPool2d((2,2),(2,1),(0,1)))
        
        # # group.add_module('conv3_5',myConv(3,128,256,1,leaky=leaky))

        group.add_module('conv4',myConv(3,256,512,1,leaky=leaky,batchNorm=True))

        group.add_module('conv5',myConv(3,512,512,1,leaky=leaky,batchNorm=True))
        group.add_module('mpool5',nn.MaxPool2d((2,2),(2,1),(0,1)))

        group.add_module('conv_final',myConv(2,512,512,0,leaky=leaky,batchNorm=True)) 

        # # PyTorch does not support asymmetrical padding

        self.cnn = group

        self.rnn = nn.Sequential(BLR(512,nHidden,nHidden),\
            BLR(nHidden,nHidden,nClass))
        
    def forward(self,input):
        '''Inference in PyTorch Lightning'''
        convolved = self.cnn(input)

        N,C,H,W = convolved.size()

        assert H==1,'Height should be 1'
        # reshaping into a sequence of length Width
        # each term is a batch of N vectors of C=512

        # height is 1 so squeeze
        convolved = convolved.squeeze(2)

        # now length of seq = W, batch size N, and features C=512
        convolved = convolved.permute(2,0,1) 

        # Expect L*N*class
        output = self.rnn(convolved)

        log_probs = F.log_softmax(output, dim = 2)

        # Collect probabilities and indices of each image's max class
        probs, preds = log_probs.max(2)
        preds_size = torch.IntTensor([preds.size(0)] * N)

        # Convert back to string
        preds = preds.transpose(1, 0).contiguous().view(-1)
        output = self.converter.decode(preds, preds_size)

        return output

    def training_step(self, batch):
        '''
        Compute and return training loss.
            Parameters:
            - batch: output of DataLoader
            - batch_idx: integer displaying the index of this batch
        '''
        criterion = nn.CTCLoss() # using CTC-Loss

        images = batch['img']
        texts = batch['labels']
        # Pass through network
        convolved = self.cnn(images)
        N,C,H,W = convolved.size()
        assert H==1,'Height should be 1'
        convolved = convolved.squeeze(2)
        convolved = convolved.permute(2,0,1)
        output = self.rnn(convolved)

        # Get log_prob
        log_probs = F.log_softmax(output, dim = 2) # along final class dimension 
        T, N, C = log_probs.size()
        input_len = torch.LongTensor([T for i in range(N)])

        # Encoding the words
        targets, target_len = self.converter.encode(texts)
        targets= targets.view(-1).contiguous() # make into contiguous storage in memory

        # Compute the loss with CTC
        loss = criterion(log_probs, targets, input_len, target_len)

        return loss

    def validation_step(self, batch, batch_idx):
        '''Validation after each epoch'''
        images = batch['img']
        texts = np.array(batch['labels'])

        out = np.array(self.forward(images))
        
        val_acc = torch.tensor(sum(out==texts), dtype = torch.float32)
        
        self.log_dict({'val_acc': val_acc})
        
    
    def configure_optimizers(self):
        '''Define optimizer'''
        lr = self.lr
        optimizer = torch.optim.Adam(self.parameters(), lr = lr)
        return optimizer
    

In [ ]:
# PyTorch has a different max method for its tensors
test_x = torch.rand(3,5,4)
vals, idx = test_x.max(2)
idx.size(0) # 3 - batch size
print(idx)

We test our implementation of ```CRNN``` for inference. 

In [ ]:
def testCRNN_forward():
    '''Test output size of CRNN.'''
    opt = {'leaky':False,'nHidden':256,'nClass':26, 'lr':1e-3}
    model = CRNN(opt)
    input = torch.rand((64,1,32,128)) # 32*128 image, 1 color channel, batch_size=64, 512 final features
    output = model(input)
    print(output[0])
    return len(output)

testCRNN_forward() # Expect L*N*class

# 2 Training

We now do the training using `Lightning Module`

In [ ]:
model = CRNN()

# trainer = pl.Trainer(check_val_every_n_epoch = 1, gpus = 1, max_epochs = 20)

trainer = pl.Trainer(gpus = 1, max_epochs = 20)

trainer.fit(model, train_dataloader = loader_train, val_dataloaders = loader_val)

# trainer.test(loader_test)